# Classification With Decision Trees, From Start to Finish

Author: ***Soroush Ghaderi***

In this lesson we will use **scikit-learn** and **Cost Complexity Pruning** to build **Classification Tree** which uses continues and categorical data from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) to predict whether or not a paitent has [heart disease](https://archive.ics.uci.edu/ml/datasets/heart+disease).

**Classification Trees** are an exponentially useful machine learning method when you need to know how the decisions are being made. For example, if you have to justify the predictions to your boss, classification trees are a good method because each step in the decision making process is easy to understand.

In this notebook we will practice about ...

- **Importing Data**


- **Missing Data**

    - Identifying Missing Data
    - Dealing with Missing Data

- **Formatting the Data for Decision Trees**

    - Splitting data into Dependent and Independent Variables
    - One-Hot Encoding
    
- **Building a Preliminary Classification Tree**


- **Optimizing the tree with Cost Complexity Pruning**

    - Visulizing Alpha
    - Using Cross Validation to find the best value for Alpha
    
- **Building, Drawing, Interpreting and Evaluating the Final Classification Tree**

The very first thing we do is load in a bunch of modules, python, itself, just gives us a basic programming language.
These modules give us extra functionality to import the data, cleen it up and format it, and then built, evaluate and draw the classification tree.

**NOTE:** If your version of `scikit-learn` is older than 0.22.1, then the easiest thing to do is just update all of your **Anaconda** packages with the following command `conda update --all` However, if you only want to update `scikit-learn`, you can run this command `install scikit-learn=0.22.1`.

In [1]:
import pandas as pd # to load and manipulate data and for one-hot encoding
import numpy as np # to calculate the mean and standard deviation
import matplotlib.pyplot as plt # to visualize graphs
from sklearn.tree import DecisionTreeClassifier # to build classification tree
from sklearn.tree import plot_tree # to draw a classification tree
from sklearn.model_selection import train_test_split # to split data into training and testing data
from sklearn.model_selection import cross_val_score # for cross validation
from sklearn.metrics import confusion_matrix # to create a confusion matrix
from sklearn.metrics import plot_confusion_matrix # to draw confusion matrix

In [2]:
# since we have already data, we can load the file
heart = pd.read_csv("data/processed.cleveland.data", header=None)
# its also possible to link the data set on the read_csv function

Now we have data on **Heart** dataframe, let's look at first five rows of data using `head()` function.

In [3]:
# print the first 5 rows
heart.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


We see that instead of nice column names, we just have column numbers. Since nice column names would make it easier to know to format the data, let's replace the column with the following column names:
* **age**
* **sex**
* **cp**, chest pain
* **restbp**, resting blood pressure (in mm Hg)
* **chol**, serum cholestrol in mg/dl
* **fbs**, fasting blood sugar
* **restecg**, resting electrocardiographic results
* **thalach**, maximum heart rate achived
* **exang**, exercise induced angina
* **oldpeak**, ST depression induced by exercise relative to rest
* **slope**, the slope of the peak exercise ST segment
* **ca**, number of major vessels(0-3) colored by fluoroscopy
* **thal**, this is short of thalium heart scan
* **hd**, diagosis of heart disease, the prediction atribute

In [4]:
# channge the column numbers to column names
heart.columns = ["age", "sex", "cp", "restbp", "chol", "fbs", "restecg", "thalach",
                 "exang", "oldpeak", "slope", "ca", "thal", "hd"]
# now print the first five rows of dataframe with new names
heart.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


We have replaced the column numbers with nice, easy to remember names. Now that we have the data in a dataframe called **heart**, we are ready to identity and deal with **Missing Data**

# Missing Data Part 1: Identifying Missing Data
Unfortunately the biggest part of any data analysis project is making sure that data is correctly formatted and fixing it when it is not. The first aprt of this procces is identifying and dealing with **Missing Data**
**Missing Data** is simply a blank space, or a surrogate value like **NA**, that indicates that we failed to collect data for one of features. For example if we forget to ask someone's age, or forget to write it down, then we would have a blank space in the dataset for that person's **age**.
There are two main ways to deal with missing data:
1. We can remove the rows that contain missing data from dataset. This is relatively easy to do, but it wastes all of the other values that we collected. How a big of a waste this is depends on how important missing values is for classification. For example, if we are missing a value for **Age** and **Age** is not useful for classifying if people have heart disease or not, then it would be a shame to throw out all of someone's data just because we do not have their **Age**
2. We can **impute** the values that are missing. In this context **impute** is just a fancy way of saying "we can make an educated guess about about what the value should be". Continuing our example where we are missing a value for **Age**. instead of throwing out the entire row of data we can fill the missing value with the average age or the median age, or use some other, more sophisticated approach , to guess at an appropriate value.

In this section , we focus on identifying missing values in the dataset

First, let's see what sort of data is in each column

In [5]:
# dtypes tells us the data type for each column
heart.dtypes

age        float64
sex        float64
cp         float64
restbp     float64
chol       float64
fbs        float64
restecg    float64
thalach    float64
exang      float64
oldpeak    float64
slope      float64
ca          object
thal        object
hd           int64
dtype: object

We see that they are almost all `float64`, however, two columns, **ca** and **thal**, have the `object` type and one column, **hd** has `int64`.
The fact that the **ca** and **thal** columns have `object` data types suggests there is something funny going on in them. `object` datatypes are used when there are mixtures of things, like a mixture of numbers and letters. In theory, both **ca** and **thal** should just have a few values representing different categories, so let's investigate what's going on by printing out their unique values, we'll start with **ca**:

In [6]:
# print out unique values in the column called 'ca'
heart["ca"].unique()

array(['0.0', '3.0', '2.0', '1.0', '?'], dtype=object)

In [7]:
# print out unique values in the column called 'thal'
heart["thal"].unique()

array(['6.0', '3.0', '7.0', '?'], dtype=object)

Both columns contains a mixture of numbers, and question marks, which represent missing values
now we identified missing values, we need to dealing with them.

# Missing Data Part 2: Dealing With Missing Data
Since Scikit-learn's classification trees do not support datasets with missing values, we need to figure out what to do with question marks. we can either delete these paitents from the training dataset, or impute values for the missing data. First let's see how many rows contain missing values.

In [8]:
# print the number of rows that contain missing values
len(heart[(heart["ca"] == "?") | (heart["thal"] == "?")])

6

Since only 6 rows have missing values, let's look at them.

In [9]:
# print out the 6 rows that contain missing value
heart.loc[(heart["ca"] == "?") | (heart["thal"] == "?")]

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
87,53.0,0.0,3.0,128.0,216.0,0.0,2.0,115.0,0.0,0.0,1.0,0.0,?,0
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,?,3.0,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,?,7.0,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,?,7.0,0
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


Now let's count the number of rows in the full dataset.

In [10]:
len(heart)

303

So **6** of the **303** rows , or **2%** contain missing values. Since **303 - 6 = 297**, and **297** is plenty of data to build a classification tree, we will reomve the rows with missing values, rather than try to impute their values.

We remove the rows with missing values by selecting all of the rows that do not contain question marksin either the **ca** and **thal** columns:

In [11]:
# save rows without missing values in another dataframe
heart_without_missing = heart[(heart["ca"] != "?") & (heart["thal"] != "?")]
heart_without_missing.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Or we can findout index of all missing values by `index()` function and use `drop()` function to drop these rows.

In [12]:
# another way to remove (drop) missing values
index_of_missing_values = heart.index[(heart["ca"] == "?") | (heart["thal"] == "?")]
heart.drop(index=index_of_missing_values, axis= 0).head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Since we drop rows missing values `heart_without_missing` dataframe should have **297** rows.

In [13]:
len(heart_without_missing)

297

We can also make sure that we dont have missing values. We can check previous function to check out if they have question marks on unique values

In [14]:
# print out unique values in the column called 'thal'
heart_without_missing["thal"].unique()

array(['6.0', '3.0', '7.0'], dtype=object)

In [15]:
# print out unique values in the column called 'ca'
heart_without_missing["ca"].unique()

array(['0.0', '3.0', '2.0', '1.0'], dtype=object)

# Format Data Part 1: Split the Data into Dependent and Independent  Variables

Now that we have taken care of the missing data, we are ready to start formattig the data for making a **Classification Tree**.
The first steo is to split data into two parts:
1. The columns of data that we will make classifications.
2. The column of data that we want to predict.

We will use th conventional of `X` (capital ***X***) to represent the columns of data that we will use to make classifications and `y` (lower case ***y***) to represent the thing we want to predicct **hd**(Heart Disease).

In [16]:
X = heart_without_missing.drop(["hd"], axis= 1) # Altrenatively: X = heart_without_missing.iloc[:,:-1]
X.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0


In [17]:
y = heart_without_missing["hd"]
y.head()

0    0
1    2
2    1
3    0
4    0
Name: hd, dtype: int64

# Format the Data Part 2: One-Hot Encoding
Now, that we have split the dataframe into two pieces, `**X**`, which contains the data we will use to predict classifications, and `**y**`, which contians the known classification in our training dataset, we need to take a closer look at the variables in `**X**`. The list bellow tells us what each variable represents and the type of data (**float** or **categorical**), it should contain:
* **age, Float**
* **sex, Category**
 * 0 = female
 * 1 = male
* **cp**, chest pain, **Category**
 * 1 = typical angina
 * 2 = physical angina
 * 3 = non-anginal pain
 * 4 = asymptomic
* **restbp**, resting blood pressure (in mm Hg), **Float**
* **chol**, serum cholestrol in mg/dl, **Float**
* **fbs**, fasting blood sugar, **Category**
 * 0 = >= 120 mg/dl
 * 1 = < 120 mg/dl
* **restecg**, resting electrocardiographic results, **Category
 * 1 = normal
 * 2 = having ST-T wave abnormality
 * 3 = showing probableor definite left ventricular hypertrophy
* **thalach**, maximum heart rate achived, **Float**
* **exang**, exercise induced angina, **Category**
 * 0 = no
 * 1 = yes
* **oldpeak**, ST depression induced by exercise relative to rest, **Float**
* **slope**, the slope of the peak exercise ST segment, **Category**
 * 1 = upsloping
 * 2 = flat
 * 3 = downsloping
* **ca**, number of major vessels(0-3) colored by fluoroscopy, **Float**
* **thal**, this is short of thalium heart scan,**Category**
 * 3 = normal
 * 6 = fixed defect
 * 7 = reversable defect

 Now just to review, let's look at the data types in **X** to remember how python seeing the data right now.

In [18]:
X.dtypes

age        float64
sex        float64
cp         float64
restbp     float64
chol       float64
fbs        float64
restecg    float64
thalach    float64
exang      float64
oldpeak    float64
slope      float64
ca          object
thal        object
dtype: object

So we see that **age, restbp, chol, thalach** are all `float64`, which is good, because we want whem to ber floating point numbers. All of the other columns, however, need to be inspected to make sure they only contain reasonable valuesm and some of them need to change. this is because, while **scikit-learn Decision Trees** natively support continous data, like resting blood preasure (**restbp**) and maximum heart rate (**thalach**), they do not natively support categorical data, like chest pain (**cp**), which contain 4 different categories. Thus, in order to use categorical data with **scikit-learn Decision Trees**, we have to use a trick that converts a column of categorical data into multiple columns of binary values. This trick is called **One-Hot Encoding**.

At this point you may be wondering "what's wrong with treating categorical data like continuous data?" To answer that question, let's look at an example:
For th **cp** (chest pain) column, we have 4 options:
 * 1 = typical angina
 * 2 = physical angina
 * 3 = non-anginal pain
 * 4 = asymptomic
 
If we treated these values, 1,2,3 and 4, like continuous data, then we would assume that 4, which means "asymptomic", is more simmilar to 3, which means "non-anginal pain" , than it is to 1 or 2, which are other types of chest pain. That means decision tree would be more likely to cluster the paitents with 4s and 3s together than the paitents with 4s and 1s together. In contrast, if we treat these numbers like categorical data, then we treat eeach one as a separate category that is no more or less similar to any of the other categories. Thus, the likelihood of clustering paitents with 4s with 3s is the same clustering 4s with 1s, and that approach is more reasonable.
Now let's inspect and, if needed, convert the columns that contain categocial and integer data into the corret datatypes, We'll start with **cp**(chest pain) ny inspecting all of its unique values:

In [19]:
X["cp"].unique()

array([1., 4., 3., 2.])

So, good news is that **cp** only contains the values it is supposed to contain, so we will convert it, using **One-Hot Encoding**, into a series of columns that only contains **0s** and **1s**

**NOTE:** There are different ways to do **One-Hot Encoding** in python. Two of the more popular methods are `ColumnTransformer()` (from **scikit-learn**) and `get_dummies()` (from **pandas**), and the both methods have pros and cons. `ColumnTransformer()` has very cool feature where it creates and persistent function that can validate data that you get in the future. For example, if you build you **Decision Tree** using a categorical variable **favorite color** that has **red, blue** and **green** options, then `ColumnTransformer()` can remember those options and later on when your **Decision Tree** is being used in a production system, if someone says their favorite color is **orange**, then `ColumnTransformer()` can throw an error or handle the situation in some other nice way. The downside of `ColumnTransformer()` is that it turns your data into an array and looses all of the column names, making it harder to verify that your usage of `ColumnTransformer()` worked as you intended it to. In contrast, `get_dummies()` leaves your your data in a dataframe and retains the column names, making it much easier to verify that it worked as intended. However, it does not have the persistant bihavior that `ColumnTransformer()` has. so, for the sake of learning learning how **One-Hot Encoding** works. 

Before we use `get_dummies()` to encode data a better approach is to change values of each series that we want to encode, for example, In **cp** (chest pian) we have 4 different values that listed above, we can change their values (1, 2, 3, 4) to (typical_angina, physical_angina, non_anginal_pain, asymptomic). We have multiple options for this transformation, `map()` and `replace()`. Both of them have the same syntax and works on `pandas.Series`. We have to do this transformation to other columns for better visualization and better column names.

In [67]:
X["cp"].map({1: "typical_angina", 2: "physical_angina", 3:"non_anginal_pain", 4:"asymptomic"})

0        typical_angina
1            asymptomic
2            asymptomic
3      non_anginal_pain
4       physical angina
             ...       
297          asymptomic
298      typical_angina
299          asymptomic
300          asymptomic
301     physical angina
Name: cp, Length: 297, dtype: object